# Day 14: Regolith Reservoir

## Approach

1. Turn paths into a grid pattern of ints. Use the int value to determine type of material: 0 = air, 1 = rock, 2 = sand
2. Will need to examine the bounds fo the path to work out what size of grid we're working with. It's not zero,zero set and we need to know when sand falls off the bounds of this -> i.e. falling into the abyss
3. We need to track sand as it falls. If it falls into the abyss then it doesn't get included in our total.

Note: sand input always appears to be 500,0 - but should probably parameterise this.

## Part 2
Going to be easier to modify in place, so have taken a copy of the notebook.


In [5]:
from IPython.display import clear_output
import time


testInput = """498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9"""

testSandInput = (500,0)

class Cave:
    def __init__(self, pathsInput:str, sandPourPoint:tuple[int,int]):
        self.minX = 1000000 #a very large numbers
        self.maxX = 0
        self.minY = 1000000 #a very large numbers
        self.maxY = 0
        self.grid = [] #grid will be offset by minX,maxX
        self.sandPourPoint = sandPourPoint
        self.minX = min(self.minX,sandPourPoint[0])
        self.maxX = max(self.maxX,sandPourPoint[0])
        self.minY = min(self.minY,sandPourPoint[1])
        self.maxY = max(self.maxY,sandPourPoint[1])

        #parse the input
        paths = []
        for r in pathsInput.splitlines():
            path = []
            chunks = r.split(' -> ')
            for c in chunks:
                coords = c.split(',')
                x = int(coords[0])
                y = int(coords[1])
                self.minX = min(self.minX,x)
                self.maxX = max(self.maxX,x)
                self.minY = min(self.minY,y)
                self.maxY = max(self.maxY,y)
                path.append((x,y))
            paths.append(path)

        #We now need to add a floor... but unsure how wide it should be... max 45degrees: so as wide as max height
        floorY = self.maxY + 2
        halfWidth =  floorY + 10 #easier to add on some contigency than worry about out-by-one errors
        self.maxY = floorY
        floorPath = [(self.sandPourPoint[0]-halfWidth,floorY),(self.sandPourPoint[0]+halfWidth,floorY)]
        paths.append(floorPath)
        self.minX = min(self.minX,self.sandPourPoint[0]-halfWidth)
        self.maxX = max(self.maxX,self.sandPourPoint[0]+halfWidth)

        #we now know the bounds of our grid
        print('Field of play: '+str(self.minX)+','+str(self.minY)+':'+str(self.maxX)+','+str(self.maxY))

        self.gridWidth = self.maxX - self.minX
        self.gridHeight = self.maxY - self.minY      

        #build an empty grid
        for x in range(self.minX, self.maxX + 1, 1):
            xrow = []
            for y in range(self.minY, self.maxY +1, 1):
                xrow.append(0)
            self.grid.append(xrow)
        

        #populate the rock
        for path in paths:
            x = -1
            y = -1
            for point in path:
                if x == -1: #first point
                    x = point[0]
                    y = point[1]
                else:
                    dx = point[0] - x
                    dy = point[1] - y
                    #only one of dx or dy should be non-zero
                    if dx!=0:
                        #draw path in x direction
                        #need to know direction
                        step = int(dx / abs(dx))
                        gridStart = x - self.minX
                        gridFinish = point[0] + (1 * step) - self.minX #out by one errors on the boundaries
                        gridY = y - self.minY
                        for gridX in range(gridStart,gridFinish,step):
                            self.grid[gridX][gridY] = 1
                    else:
                        #draw path in y direction
                        step = int(dy / abs(dy))
                        gridStart = y - self.minY
                        gridFinish = point[1] + (1 * step) - self.minY
                        gridX = x - self.minX
                        for gridY in range(gridStart,gridFinish,step):
                            self.grid[gridX][gridY] = 1
                    #update last point
                    x = point[0]
                    y = point[1]
                    #print('---')
                    #print(self)

    def __str__(self):
        #print columns 0> and rows 0down
        outputStr = ''
        for y in range(self.gridHeight+1):
            rowStr = ''
            for x in range(self.gridWidth+1):
                match self.grid[x][y]:
                    case 0:
                        rowStr = rowStr + '.'
                    case 1:
                        rowStr = rowStr + '#'
                    case 2:
                        rowStr = rowStr + 'o'
            outputStr = outputStr + '\n' + rowStr
        return outputStr


    def outOfBounds(self, pos:tuple[int,int]):
        #check if the position is out of bounds - used to check if the sand will fall forever.
        return pos[0] < 0 or pos[0] > self.gridWidth or pos[1] < 0 or pos[1] > self.gridHeight

    def pourSand(self):
        #pour sand one chunk at a time
        abyss = False
        sandCounter = -1 #sandCounter will always over count 1, because it will also count the sand that falls to they abyss... so we coutneract that by starting at -1
        while not abyss: 
            #loop of each bit of sand, so we stat with sand at the sand pouring point
            sandCounter += 1
            print(str(sandCounter) + ' units of sand')
            sandPourGridPosition = (self.sandPourPoint[0]-self.minX,self.sandPourPoint[1]-self.minY)
            sandPos = sandPourGridPosition
            sandMoving = True
            while sandMoving and not abyss: 
                #A unit of sand always falls down one step if possible
                downOne = (sandPos[0],sandPos[1]+1)
                diagLeft = (sandPos[0]-1,sandPos[1]+1)
                diagRigth = (sandPos[0]+1,sandPos[1]+1)
                if self.outOfBounds(downOne):
                    abyss = True
                elif self.grid[downOne[0]][downOne[1]] == 0:
                    sandPos = downOne
                elif self.outOfBounds(diagLeft):
                    abyss = True
                elif self.grid[diagLeft[0]][diagLeft[1]] == 0:
                    sandPos = diagLeft
                elif self.outOfBounds(diagRigth):
                    abyss = True
                elif self.grid[diagRigth[0]][diagRigth[1]] == 0:
                    sandPos = diagRigth 
                else:
                    #sand has stopped
                    self.grid[sandPos[0]][sandPos[1]] = 2
                    sandMoving = False
                    if sandPos == sandPourGridPosition:
                        sandCounter += 1
                        abyss = True


        print(str(sandCounter)+' units of sand')

#test
testCave = Cave(testInput, testSandInput)
print(testCave)
testCave.pourSand()


Field of play: 479,0:521,11

...........................................
...........................................
...........................................
...........................................
...................#...##..................
...................#...#...................
.................###...#...................
.......................#...................
.......................#...................
...............#########...................
...........................................
###########################################
0 units of sand
1 units of sand
2 units of sand
3 units of sand
4 units of sand
5 units of sand
6 units of sand
7 units of sand
8 units of sand
9 units of sand
10 units of sand
11 units of sand
12 units of sand
13 units of sand
14 units of sand
15 units of sand
16 units of sand
17 units of sand
18 units of sand
19 units of sand
20 units of sand
21 units of sand
22 units of sand
23 units of sand
24 units of sand
25 units of sand
26 units of

In [7]:
#real fun
input = open('day14input.txt').read()
realCave = Cave(input, testSandInput)
print(realCave)
realCave.pourSand()
#print(realCave)


Field of play: 317,0:683,173

...............................................................................................................................................................................................................................................................................................................................................................................
...............................................................................................................................................................................................................................................................................................................................................................................
..........................................................................................................................................................................................................................................